# Best practices when programming a deep learning model
> A tutorial for best practices when programming a deep learning model.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

best practices when programming a deep learning model. These practices mostly refer to how we can write organized, modularized, and extensible python code.

Most of them aren’t exclusive for machine learning applications but can be utilized on all sorts of python projects. But here we will see how we can apply them in deep learning using a hands-on programming approach.

Machine learning code is ordinary software and should always be treated as one. Therefore, as ordinary code it has a project structure, a documentation and design principles such as object-oriented programming.

Devops for ML
“This key chapter puts the “engineering” in data engineering. DevOps practices allow us to build reliable, reproducible systems. One of the principles you will see repeated throughout the book is tracking everything in source control and deploying everything automatically.”

Excerpt From
Data Engineering on Azure Riscutia, Vlad;




===== https://theaisummer.com/deep-learning-production/



It’s time. You spent hours reading research papers, experimenting on different data, testing the accuracy of different models but you got it. After training your model locally, you’ve seen some pretty awesome results and you’re convinced that your model is ready to go. What’s next?
The next step is to take your experimentation code and migrate it into an actual python project with proper structure, unit tests, static code analysis, parallel processing etc.


In terms of software, the flow is something like this: the user uploads the image to the browser, the browser sends the image to our backend, the UNet predicts the segmented image and we send the image back to the user’s browser, where it is rendered.

￼
After watching the above system, there are many questions that come in our mind:
* What will happen if many users ask the model at the same time?
* What will happen if for some reason the model crashes?
* If it contains a bug we haven’t previously seen?
* If the model is too slow and the user has to wait for too long to get a response?
* And how can we update our model?
* Can we retrain it on new data?
* Can we get some feedback if the model performs well?
* Can we see how fast or slow the model is?
* How much memory does it use?
* Where do we store the model weights?
* What about the training data?
* What if the user sends a malicious request instead of a proper image?

I can go on forever but I’m sure you get my point. Most of these questions will be answered in the following articles. That’s why in most real-life cases, the final architecture will be something like this:

￼

Ok, we won’t go that far. That’s a whole startup up there. But we will cover many points focusing on optimizing the software at first and touching some of these parts on our latest articles.

Software design principles
But to develop our software, we need to have in mind a rough idea of our end goal and what we want to achieve. Every system should be built based on some basic principles:
* Separation of concerns: The system should be modularized into different components with each component being a separate maintainable, reusable and extensible entity.
* Scalability: The system needs to be able to scale as the traffic increases
* Reliability: The system should continue to be functional even if there is software of hardware failure
* Availability: The system needs to continue operating at all times
* Simplicity: The system has to be as simple and intuitive as possible
Given the aforementioned principles, let’s discuss the above image:
Each user sends a request to our backend. To be able to handle all the simultaneous requests, first we need a Load Balancer (LB) to handle the traffic and route the requests to the proper server instance.
We maintain more than one instance of the model in order to achieve both scalability and availability. Note that these might be physical instances, virtual machines or docker containers and they are organized by a distributed system orchestrator (such as Kubernetes)
As you can probably see, each instance may have more than one threads or parallel processes. We need that to make sure that the model is fast enough to return a response in real time.
A major problem most machine learning systems face is retraining because in order for the model to stay up to date, you should constantly update the model weights based on the new data. In our case, this pipeline consists of:
* A database (DB) to save the requests, the responses and all relative data
* A message queue (MQ) to send them in the database in an asynchronous manner ( to keep the system reliable and available)
* Data jobs to preprocess and transform the data in the correct format so that can be used from our model
* Retrainer instances that execute the actual training based on the saved data

After retraining the model, the new version will gradually replace all the UNet instances. That way we build deep learning models versioning functionality, where we always update the model with the newest version.
(Really? In this case, we need to make sure the newest version is the best, how ?)

Finally, we need some sort of monitoring and logging to have a complete image of what’s happening in our system, to catch errors quickly and to discover bottlenecks easily.


Get ready for Software Engineering
1. understand the whys of a modern deep learning system.

The second one requires some mental effort because deep learning architectures are rather complex systems. But the main thing that you need to remember are the 5 principles as they will accompany us down the road.
We are all set up and ready to begin our journey towards deploying our UNet model and serve it to our users. As I said in the beginning, brace yourself to go deep into programming deep learning apps, to dive into details that you probably never thought of and most of all to enjoy the process.


https://github.com/donnemartin/system-design-primer

https://theaisummer.com/best-practices-deep-learning-code/

Project Structure
One very important aspect when writing code is how you structure your project. A good structure should obey the “Separation of concerns” principle in terms that each functionality should be a distinct component. In this way, it can be easily modified and extended without breaking other parts of the code. Moreover, it can also be reused in many places without the need to write duplicate code.

The way I like to organize most of my deep learning projects is something like this :

￼

And that, of course, is my personal preference. Feel free to play around with this until you find what suits you best.


Python modules and packages

Notice that in the project structure that I presented, each folder is a separate module that can be imported in other modules just by doing “import module”. Here, we should make a quick distinction between what python calls module and what package. A module is simply a file containing Python code. A package, however, is like a directory that holds sub-packages and modules. In order for a package to be importable, it should contain a init.py file ( even if it’s empty). That’s not the case for modules. Thus, in our case each folder is a package and it contains an “init” file.

In our example, we have 8 different packages:

1. configs: in configs we define every single thing that can be configurable and can be changed in the future. Good examples are training hyperparameters, folder paths, the model architecture, metrics, flags.
A very simple config is something like this:
CFG = {
   "data": {
       "path": "oxford_iiit_pet:3.*.*",
       "image_size": 128,
       "load_with_info": True
   },
   "train": {
       "batch_size": 64,
       "buffer_size": 1000,
       "epoches": 20,
       "val_subsplits": 5,
       "optimizer": {
           "type": "adam"
       },
       "metrics": ["accuracy"]
   },
   "model": {
       "input": [128, 128, 3],
       "up_stack": {
           "layer_1": 512,
           "layer_2": 256,
           "layer_3": 128,
           "layer_4": 64,
           "kernels": 3
       },
       "output": 3
   }
}
2. dataloader is quite self-explanatory. All the data loading and data preprocessing classes and functions live here.
3. evaluation is a collection of code that aims to evaluate the performance and accuracy of our model.
4. executor: in this folder, we usually have all the functions and scripts that train the model or use it to predict something in different environments. And by different environments I mean: executors for GPUs, executors for distributed systems. This package is our connection with the outer world and it’s what our “main.py” will use.
5. model contains the actual deep learning code (we talk about tensorflow, pytorch etc)
6. notebooks include all of our jupyter/colab notebooks in one place.
7. ops: this one is not always needed, as it includes operations not related with machine learning such as algebraic transformations, image manipulation techniques or maybe graph operations.
8. utils: utilities functions that are used in more than one places and everything that don’t fall in on the above come here.
Now that we have our project well structured and all, we can begin to see how our code should look like on a lower level.

